In [1]:
# Import the functions
from trajectory_functions import *

# Import other required libraries
import numpy as np
import torch
import matplotlib.pyplot as plt
from IPython.display import display

IndentationError: unexpected indent (trajectory_functions.py, line 877)

In [ ]:

## 2. Load Sparse Autoencoder Model (if available)

```python
# If you have a model, load it here
model_path = "path/to/your/model.pt"  # Update with your model path

# Load the model
try:
    model = load_model(model_path)
except Exception as e:
    print(f"Could not load model: {e}")
    print("Continuing without model...")
    model = None
```

In [ ]:

## 3. Generate Different Types of Trajectories

### 3.1 Simple Oscillator

```python
# Generate a damped oscillator
oscillator_data = generate_oscillator(
    n_points=200, 
    period=5.0, 
    amplitude=2.0, 
    phase_shift=0.5, 
    damping=0.1,
    dimensions=3
)

# Simple 3D visualization of the trajectory
trajectory = oscillator_data['trajectory']
time_points = oscillator_data['time_points']

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], 'b-')
ax.set_title('3D Damped Oscillator')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()
```

In [ ]:
### 3.2 Lorenz Attractor

```python
# Generate a Lorenz attractor trajectory
lorenz_data = generate_lorenz(
    n_points=1000,
    total_time=50.0,
    sigma=10.0,
    rho=28.0,
    beta=8/3,
    noise_level=0.0
)

# Visualize the Lorenz attractor
lorenz_trajectory = lorenz_data['trajectory']

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot(lorenz_trajectory[:, 0], lorenz_trajectory[:, 1], lorenz_trajectory[:, 2], 'r-')
ax.set_title('Lorenz Attractor')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()
```

In [ ]:
### 3.3 Double Pendulum

```python
# Generate a double pendulum trajectory
pendulum_data = generate_double_pendulum(
    n_points=500,
    total_time=15.0,
    initial_state=[np.pi/2, 0, np.pi/4, 0]  # Starting angles and velocities
)

# Visualize pendulum bob positions over time
pendulum_trajectory = pendulum_data['trajectory']
pendulum_time = pendulum_data['time_points']

# Extract positions of both bobs
x1, y1 = pendulum_trajectory[:, 0], pendulum_trajectory[:, 1]
x2, y2 = pendulum_trajectory[:, 2], pendulum_trajectory[:, 3]

# Plot the paths of both pendulum bobs
plt.figure(figsize=(10, 8))
plt.plot(x1, y1, 'b-', label='Bob 1')
plt.plot(x2, y2, 'r-', label='Bob 2')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.legend()
plt.title('Double Pendulum Path')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()
```

In [ ]:
## 4. Analyze Feature Activations (if model is loaded)

```python
if model is not None:
    # Compute activations for the Lorenz attractor
    lorenz_activations, _ = compute_activations(model, lorenz_trajectory)
    
    # Find top features
    top_features = find_top_features(lorenz_activations, top_n=5)
    print(f"Top activated features for Lorenz attractor: {top_features}")
    
    # Visualize the top feature
    feature_idx = top_features[0]
    fig, axes = plot_activation_along_trajectory(
        lorenz_trajectory, lorenz_activations, feature_idx=feature_idx,
        time_points=lorenz_data['time_points'], equation=lorenz_data['equation'],
        figsize=(12, 8), cmap='viridis'
    )
    plt.tight_layout()
    plt.show()
```

In [ ]:
## 5. Use the Quick Visualization Function

```python
# Quickly visualize a spiral with activations
spiral_viz = quick_trajectory_visualization(
    generate_spiral,
    model=model,
    feature_idx=0 if model is None else top_features[0],
    n_points=200,
    radius_start=0.5,
    radius_end=3.0,
    revolutions=5
)
```

In [ ]:
## 6. Create Interactive Interface

```python
# Create the interactive visualization widget
interactive_widget = create_interactive_visualization(model)

# The widget will be displayed and allow you to explore different trajectories
# and how they activate features in your model
```

In [ ]:

## 7. Advanced: Customizing Trajectories

```python
# You can create custom trajectories by modifying the generator functions
# For example, let's create a combination of oscillators:

def generate_combined_oscillator(n_points=200, periods=[5.0, 8.0], amplitudes=[1.0, 0.5]):
    """Generate a combination of oscillators with different periods"""
    time_points = np.linspace(0, max(periods)*2, n_points)
    trajectory = np.zeros((n_points, 2))
    
    for i, (period, amplitude) in enumerate(zip(periods, amplitudes)):
        omega = 2 * np.pi / period
        trajectory[:, 0] += amplitude * np.sin(omega * time_points)
        trajectory[:, 1] += amplitude * np.cos(omega * time_points)
    
    equation = "Combined oscillators with periods " + ", ".join([f"{p:.1f}" for p in periods])
    
    return {
        'trajectory': trajectory,
        'time_points': time_points,
        'equation': equation
    }

# Visualize the combined oscillator
combined_data = generate_combined_oscillator(periods=[5.0, 8.0, 12.0], amplitudes=[1.0, 0.5, 0.25])
combined_trajectory = combined_data['trajectory']

plt.figure(figsize=(10, 8))
plt.plot(combined_trajectory[:, 0], combined_trajectory[:, 1], 'g-')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.title('Combined Oscillators')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

# If you have a model, you can analyze activations for this custom trajectory too
if model is not None:
    combined_activations, _ = compute_activations(model, combined_trajectory)
    top_features = find_top_features(combined_activations, top_n=5)
    print(f"Top activated features for combined oscillator: {top_features}")
```